In [36]:
import pandas as pd
import numpy as np
champion_user_df = pd.read_csv('datasets/csv/champion_user_df_tfidf3.csv', sep=',', engine='python')
champion_position_df = pd.read_csv('datasets/csv/champion_positions.csv', sep=",", engine='python')
champion_user_position_df = champion_user_df.merge(champion_position_df, left_on='_champion_id', right_on='champion')
champion_user_position_df = champion_user_position_df.drop(columns=['Unnamed: 0','champion'])
#filter to top position
top_champion_user_df = champion_user_position_df[champion_user_position_df['position'] == 'Top']

In [37]:
matrix_pc = top_champion_user_df.pivot_table(values='_play_count', index='_champion_id', columns='user_name')
matrix_tfidf = top_champion_user_df.pivot_table(values='tfidf', index='_champion_id', columns='user_name')

In [3]:
top_champion_user_df

,_championPoints,_champion_id,_champion_key,_play_count,_win_rate,user_name,total_play,win_rate,ranking,tier_rank,playcount_sum,tf,champ_count,idf,tfidf,position
0,14780,Jayce,126,3,0.33,왕 난,36,0.528,17.50,Gold 2,23,0.130435,1218,1.193106,0.155623,Top
2,21914,Jayce,126,19,0.47,티모와함께라면,123,0.528,17.26,Gold 2,74,0.256757,1218,1.193106,0.306338,Top
4,14385,Jayce,126,21,0.48,00 03 23,155,0.561,14.42,Gold 1,161,0.130435,1218,1.193106,0.155623,Top
6,16959,Jayce,126,5,0.60,010223,42,0.571,17.62,Gold 2,46,0.108696,1218,1.193106,0.129685,Top
8,12031,Jayce,126,6,0.83,01048114591,85,0.518,48.40,Silver 2,71,0.084507,1218,1.193106,0.100826,Top
10,56509,Jayce,126,5,0.80,02 27 2018,27,0.481,17.55,Gold 2,19,0.263158,1218,1.193106,0.313975,Top
12,221292,Jayce,126,30,0.53,0H비,192,0.474,29.13,Gold 3,135,0.222222,1218,1.193106,0.265135,Top
14,27543,Jayce,126,27,0.74,1267456887,160,0.581,14.54,Gold 1,124,0.217742,1218,1.193106,0.259789,Top
16,8113,Jayce,126,10,0.40,1396윤,178,0.522,33.67,Gold 4,102,0.098039,1218,1.193106,0.116971,Top
18,18475,Jayce,126,5,0.60,13일금,114,0.570,17.26,Gold 2,86,0.058140,1218,1.193106,0.069367,Top


# champion champion relationship

In [38]:
champion_winrate_df = pd.read_csv('datasets/csv/champion_winrate.csv', sep=',', engine='python')

In [39]:
top_champion_user_df1 = top_champion_user_df.copy()
top_champion_user_df1 = top_champion_user_df1.merge(champion_winrate_df, left_on='_champion_id', right_on='champion')
top_champion_user_df1['win_rate_y'] = top_champion_user_df1.apply(lambda x: float(x['win_rate_y'].strip('%'))/100, axis=1)

In [40]:
alpha = 0.5
top_champion_user_df1['score_1'] = top_champion_user_df1.apply(lambda row: (alpha * row['win_rate_y']) + ((1-alpha)*row['tfidf']), axis=1)

In [41]:
matrix_pc = top_champion_user_df1.pivot_table(values='score_1', index='_champion_id', columns='user_name')

In [42]:
matrix_pc_cosine = matrix_pc.copy().fillna(0)

# Import cosine_score 
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity matrix using the dummy ratings matrix
pc_cosine = cosine_similarity(matrix_pc_cosine, matrix_pc_cosine)

#Convert into pandas dataframe 
pc_cosine = pd.DataFrame(pc_cosine, index=matrix_pc_cosine.index, columns=matrix_pc_cosine.index)

In [43]:
#flatten
pc_cosine_sim_flat = pc_cosine.copy()
pc_cosine_sim_flat = pc_cosine_sim_flat.where(np.triu(np.ones(pc_cosine_sim_flat.shape)).astype(np.bool))
del pc_cosine_sim_flat.columns.name
del pc_cosine_sim_flat.index.name
pc_cosine_sim_flat = pc_cosine_sim_flat.stack().reset_index()
pc_cosine_sim_flat.columns = ['From','To','Weight']
pc_cosine_sim_flat[pc_cosine_sim_flat['From'] != pc_cosine_sim_flat['To']].sort_values(by='Weight', ascending = False)
pc_cosine_sim_flat = pc_cosine_sim_flat[pc_cosine_sim_flat['From'] != pc_cosine_sim_flat['To']].sort_values(by='Weight', ascending = False)
pc_cosine_sim_flat

,From,To,Weight
472,Irelia,Yasuo,0.265260
55,Akali,Irelia,0.219989
1018,Vayne,Yasuo,0.219309
913,Riven,Yasuo,0.214747
458,Irelia,Riven,0.202518
988,Sylas,Yasuo,0.190068
1027,Vladimir,Yasuo,0.187367
87,Akali,Yasuo,0.173342
464,Irelia,Sylas,0.170155
172,Darius,Yasuo,0.167384


# construct Nodes dataset

In [44]:
champion_lists = top_champion_user_df1.drop_duplicates('_champion_id')
champion_lists = champion_lists[['_champion_id']].copy()
champion_lists['attribute'] = 2
champion_lists.columns = ['Label','Attribute']
#nodes = pd.DataFrame()
#user_lists

In [45]:
champion_lists

,Label,Attribute
0,Jayce,2
1218,Vladimir,2
3341,Renekton,2
4273,Yasuo,2
6958,Kennen,2
7401,Vayne,2
11481,Irelia,2
13383,Pantheon,2
14451,Sylas,2
16314,Hecarim,2


In [46]:
nodes = pd.DataFrame()
nodes = nodes.append([champion_lists], ignore_index=True)
nodes.index +=1
nodes.reset_index(level=0, inplace=True)
nodes = nodes.rename(columns={'index': 'Id'})
nodes

,Id,Label,Attribute
0,1,Jayce,2
1,2,Vladimir,2
2,3,Renekton,2
3,4,Yasuo,2
4,5,Kennen,2
5,6,Vayne,2
6,7,Irelia,2
7,8,Pantheon,2
8,9,Sylas,2
9,10,Hecarim,2


# construct Edges dataset

In [47]:
def construct_edge(cosine_data):
    cosine_data['Source'] = cosine_data['From'].map(nodes.set_index('Label')['Id'])
    cosine_data['Target'] = cosine_data['To'].map(nodes.set_index('Label')['Id'])
    cosine_data['Type'] = 'Undirected'
    cosine_data = cosine_data[['Source', 'Target', 'Type', 'Weight']]
    return cosine_data

In [54]:
#champion champion relationship edge
champion_champion = construct_edge(pc_cosine_sim_flat)
champion_champion

,Source,Target,Type,Weight
472,7,4,Undirected,0.265260
55,23,7,Undirected,0.219989
1018,6,4,Undirected,0.219309
913,13,4,Undirected,0.214747
458,7,13,Undirected,0.202518
988,9,4,Undirected,0.190068
1027,2,4,Undirected,0.187367
87,23,4,Undirected,0.173342
464,7,9,Undirected,0.170155
172,26,4,Undirected,0.167384


# export csv champion only

In [59]:
champion_champion = champion_champion[champion_champion['Weight'] > 0]
champion_champion_exp = champion_champion.copy()
champion_champion_exp['Source'] = champion_champion_exp['Source'].map(nodes.set_index('Id')['Label'])
champion_champion_exp['Target'] = champion_champion_exp['Target'].map(nodes.set_index('Id')['Label'])

In [60]:
champion_champion_exp[['Source', 'Target', 'Weight']].to_csv('datasets/csv/top_edges_2.csv', index=False)

In [51]:
nodes_champ = pd.DataFrame()
nodes_champ = nodes_champ.append(champion_lists, ignore_index=True)
nodes_champ.index +=1
nodes_champ.reset_index(inplace=True)
nodes_champ = nodes_champ.rename(columns={'index': 'Id'})
edges_champ = champion_champion.copy()

In [52]:
nodes_champ

,Id,Label,Attribute
0,1,Jayce,2
1,2,Vladimir,2
2,3,Renekton,2
3,4,Yasuo,2
4,5,Kennen,2
5,6,Vayne,2
6,7,Irelia,2
7,8,Pantheon,2
8,9,Sylas,2
9,10,Hecarim,2


In [53]:
nodes_champ.to_csv('datasets/csv/graph/top_nodes_champonly_test2.csv', index=False)
edges_champ.to_csv('datasets/csv/graph/top_edges_champonly_test2.csv', index=False)